# Aprendizaje Multietiqueta de Patrones Geométricos en Objetos de Herencia Cultural
# Resnet Retraining
## Seminario de Tesis II, Primavera 2022
### Master of Data Science. Universidad de Chile.
#### Prof. guía: Benjamín Bustos - Prof. coguía: Iván Sipirán
#### Autor: Matías Vergara
El objetivo de este notebook es entrenar arquitecturas de ResNet preentrenadas sobre la tarea de predecir el capítulo de cada patrón.

## Imports

In [2]:
folder_path = '../'

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
import time
import os
import copy
import pandas as pd
import math
import random
import shutil

from torch.utils.data import Dataset
from PIL import Image

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np, scipy.io
import argparse
import json

## Configuración de dispositivo

In [ ]:
CUDA_ID = 0

device = torch.device(f'cuda:{CUDA_ID}' if torch.cuda.is_available() else 'cpu')
print(f"Usando device: {torch.cuda.get_device_name(device)}")

## Configuración de datos y modelo

In [13]:
# Flags para los datos sintéticos
# Cada flag está asociada a una o más funciones de data augmentation.
# Los datos deben existir previamente 
# (se generan a partir del notebook split and augmentation)
USE_RN50 = True
DS_FLAGS = ['ref', 'rot', 'crop', 'elastic']
              # 'ref': [invertX, invertY],
              # 'rot': [rotate90, rotate180, rotate270],
              # 'crop': [crop] * CROP_TIMES,
              # 'blur': [blur],
              # 'emboss': [emboss],
              # 'randaug': [randaug],
              # 'rain': [rain],
              # 'elastic': [elastic]
                
# Las flags crop, randaug, elastic y gausblur 
# se pueden aplicar más de una vez c/u. 
# (si no están en DS_FLAGS, serán ignoradas).
CROP_TIMES = 1
RANDOM_TIMES = 1
ELASTIC_TIMES = 1

# Guardar el modelo cada SAVE_EACH épocas
SAVE_EACH = -1 # -1 to save only the best model
TRAINING_EPOCHS = 300
K = 4
k_model = 0

In [14]:
# Esta celda construye la variable data_flags, que lee DS_FLAGS de 
# la celda anterior y mapea su contenido a distintas rutas de 
# patrones, etiquetas y outputs
MAP_TIMES = {'crop': CROP_TIMES,
         'randaug': RANDOM_TIMES,
         'elastic': ELASTIC_TIMES,
}

DS_FLAGS = sorted(DS_FLAGS)
data_flags = '_'.join(DS_FLAGS) if len(DS_FLAGS) > 0 else 'base'
MULTIPLE_TRANSF = ['crop', 'randaug', 'elastic']
COPY_FLAGS = DS_FLAGS.copy()

for t in MULTIPLE_TRANSF:
    if t in DS_FLAGS:
        COPY_FLAGS.remove(t)
        COPY_FLAGS.append(t + str(MAP_TIMES[t]))
        data_flags = '_'.join(COPY_FLAGS)

patterns_path = folder_path + 'patterns/' + data_flags + '/' + str(k_model)
labels_path = folder_path + 'labels/' + data_flags + '/' + str(k_model)
if not (os.path.isdir(patterns_path) and os.path.isdir(labels_path)):
    raise FileNotFoundError("No existen directorios de datos para el conjunto de flags seleccionado. Verifique que el dataset exista y, de lo contrario, llame a Split and Augmentation")
print("Pattern set encontrado en {}".format(patterns_path))
print("Labels set encontrado en {}".format(labels_path))
OUTPUT_FILENAME = f'resnet50_K{k_model}.pth' if USE_RN50 else f'resnet18_K{k_model}.pth'

Pattern set encontrado en ../patterns/ref_rot_crop1_elastic1/0
Labels set encontrado en ../labels/ref_rot_crop1_elastic1/0


In [15]:
model_output_dir = folder_path + 'models/resnet/{}'.format(data_flags)
model_output_path = model_output_dir + OUTPUT_FILENAME
os.makedirs(model_output_dir, exist_ok=True)
print(f"El modelo resultante se guardará en {model_output_dir}")

El modelo resultante se guardará en ../models/resnet/ref_rot_crop1_elastic1/


## Transfer Learning

In [16]:
pathDataset = patterns_path + '/'

train_dataset = torchvision.datasets.ImageFolder(pathDataset + 'train', 
                                                    transform = transforms.Compose([
                                                        transforms.RandomVerticalFlip(),
                                                        transforms.RandomHorizontalFlip(),
                                                        transforms.RandomResizedCrop(224),
                                                                    transforms.ToTensor(),
                                                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                        std = [0.229, 0.224, 0.225])]))

val_dataset = torchvision.datasets.ImageFolder(pathDataset + 'val',
                                                    transform = transforms.Compose([ transforms.Resize(256),
                                                                    transforms.CenterCrop(224),
                                                                    transforms.ToTensor(),
                                                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                        std = [0.229, 0.224, 0.225])]))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)

class_names = train_dataset.classes

print(class_names)

def train_model(model, criterion, optimizer, num_epochs=30, output_path = 'model.pth', save_each = -1, patience=15):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    bad_epochs = 0
    best_epoch = 0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-' * 10)

        model.train()

        running_loss = 0.0
        running_corrects = 0.0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds ==  labels.data)
        
        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects.double() / len(train_dataset)

        print('Train Loss: {:.4f}  Acc: {:.4f}'.format(epoch_loss, epoch_acc))

        #Validation
        model.eval()
        running_loss = 0.0
        running_corrects = 0.0

        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(val_dataset)
        epoch_acc = running_corrects / len(val_dataset)
        print('Val Loss: {:.4f}  Acc: {:.4f}'.format(epoch_loss, epoch_acc))

        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            bad_epochs = 0
            best_epoch = epoch    

        else:
            bad_epochs += 1
            if bad_epochs == patience:
                print(f"Se agotó la paciencia. Mejor época: {best_epoch}.")
                break
                
        if save_each > -1 and epoch%save_each == 0:
            path = output_path.split("/")
            filename =  path[-1]
            epoch_filename =filename.split(".")[0] + "_e" + str(epoch) + "." + filename.split(".")[1]
            new_path = path[:-1]
            new_path.append(epoch_filename)
            new_path = '/'.join(new_path)
            torch.save(model.state_dict(), new_path)
            print("Saving model at epoch {} as {}".format(epoch, new_path))

            
    print('Best accuracy: {:.4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)

    return model

['circular ornaments', 'lozenge', 'pictographics', 'rectangular ornaments', 'strokes and lines', 'triangular ornaments']


In [18]:
if USE_RN50:
    model_ft = models.resnet50(pretrained=True)
else:
    model_ft = models.resnet18(pretrained=True)
num_ft = model_ft.fc.in_features

output_dim = 6
model_ft.fc = nn.Linear(num_ft, output_dim)

model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()

learning_rate = 0.001
groups = [{'params': model_ft.conv1.parameters(),'lr':learning_rate/4},
            {'params': model_ft.bn1.parameters(),'lr':learning_rate/4},
            {'params': model_ft.layer1.parameters(),'lr':learning_rate/4},
            {'params': model_ft.layer2.parameters(),'lr':learning_rate/2},
            {'params': model_ft.layer3.parameters(), 'lr':learning_rate/2},
            {'params': model_ft.layer4.parameters(),'lr':learning_rate},
            {'params': model_ft.fc.parameters(), 'lr':learning_rate}]

optimizer = torch.optim.Adam(model_ft.parameters(), lr = 0.0015)

output_path = model_output_path

# change save_each and output_path to get partial outputs
model_ft = train_model(model_ft, criterion, optimizer, num_epochs=TRAINING_EPOCHS,
                       save_each=SAVE_EACH, output_path=output_path)

# save best model
torch.save(model_ft.state_dict(), output_path)

Epoch 0/299
----------
Train Loss: 1.6840  Acc: 0.3317
Val Loss: 2.0425  Acc: 0.2564
Epoch 1/299
----------
Train Loss: 1.4316  Acc: 0.4448
Val Loss: 1.4869  Acc: 0.4744
Epoch 2/299
----------
Train Loss: 1.2990  Acc: 0.5138
Val Loss: 1.9419  Acc: 0.5256
Epoch 3/299
----------
Train Loss: 1.2186  Acc: 0.5451
Val Loss: 1.1046  Acc: 0.6410
Epoch 4/299
----------
Train Loss: 1.1337  Acc: 0.5789
Val Loss: 1.0303  Acc: 0.7051
Epoch 5/299
----------
Train Loss: 1.1065  Acc: 0.5829
Val Loss: 1.6207  Acc: 0.4872
Epoch 6/299
----------
Train Loss: 1.0296  Acc: 0.6110
Val Loss: 1.0530  Acc: 0.6795
Epoch 7/299
----------
Train Loss: 0.9849  Acc: 0.6243
Val Loss: 1.1282  Acc: 0.6154
Epoch 8/299
----------
Train Loss: 0.9407  Acc: 0.6528
Val Loss: 1.0943  Acc: 0.6795
Epoch 9/299
----------
Train Loss: 0.9088  Acc: 0.6731
Val Loss: 1.0666  Acc: 0.6923
Epoch 10/299
----------
Train Loss: 0.8562  Acc: 0.6812
Val Loss: 1.3402  Acc: 0.6410
Epoch 11/299
----------
Train Loss: 0.8268  Acc: 0.7001
Val Loss

## Testing Transfer Learning

In [19]:
model = model_output_path

pathDataset = patterns_path + '/'

test_dataset = torchvision.datasets.ImageFolder(pathDataset + 'test',
                                                    transform = transforms.Compose([ transforms.Resize(224),
                                                                    #transforms.CenterCrop(224),
                                                                    transforms.ToTensor(),
                                                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                        std = [0.229, 0.224, 0.225])]))

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)
device = ('cuda' if torch.cuda.is_available() else 'cpu')

if USE_RN50:
    model_ft = models.resnet50(pretrained=True)
else:
    model_ft = models.resnet18(pretrained=True)

output_dim = 6
model_ft.fc = nn.Linear(num_ft, output_dim)

model_ft = model_ft.to(device)

model_ft.load_state_dict(torch.load(model))
criterion = nn.CrossEntropyLoss()

model_ft.eval()
running_loss = 0.0
running_corrects = 0.0

for inputs, labels in test_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.set_grad_enabled(False):
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

epoch_loss = running_loss / len(test_dataset)
epoch_acc = running_corrects / len(test_dataset)
print('Test Loss: {:.4f}  Acc: {:.4f}'.format(epoch_loss, epoch_acc))

Test Loss: 0.8068  Acc: 0.7577
